In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
top = np.percentile(train.Purchase,99.5)
train = train[ (train['Purchase']<top) ]
#test1.head()
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)
dl = pd.DataFrame()
dl['User_ID'] = test['User_ID']
dl['Product_ID'] = test['Product_ID']

In [2]:
cat_col=["User_ID", "Product_ID"]
for var in cat_col:
    lb = preprocessing.LabelEncoder()
    full_var_data = pd.concat((train[var], test[var]), axis=0).astype('str')
    lb.fit(full_var_data)
    train[var] = lb.transform(train[var].astype('str'))
    test[var] = lb.transform(test[var].astype('str'))

In [3]:
new_gender = {'F':0, 'M':1}
train.Gender.replace(new_gender,inplace=True)
test.Gender.replace(new_gender,inplace=True)
# Get  columns whose data type is object
filteredColumns = train.dtypes[train.dtypes == np.object]
# list of columns whose data type is object
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

['Age', 'City_Category', 'Stay_In_Current_City_Years']


In [4]:
for i in listOfColumnNames:
    if i!=0:
        train=pd.get_dummies(train, columns=[i])
        test=pd.get_dummies(test, columns=[i])

In [5]:
user_id_to_category_map = {}
customer_purchase_power = train.groupby("User_ID")["Purchase"].sum()
#customer_purchase_power.median()

values = customer_purchase_power.iteritems()
for key, val in values:
    
    if val <= 143662.0:
        user_id_to_category_map[key] = 1
    if val <= 202156.0:
        user_id_to_category_map[key] = 2
    elif val <= 275408.0:
        user_id_to_category_map[key] = 3
    elif val <= 378451.0:
        user_id_to_category_map[key] = 4
    elif val <= 513874.0:
        user_id_to_category_map[key] = 5
    elif val <= 687241.0:
        user_id_to_category_map[key] = 6
    elif val <= 934097.0:
        user_id_to_category_map[key] = 7
    elif val <= 1332584.0:
        user_id_to_category_map[key] = 8
    elif val <= 2029891.0:
        user_id_to_category_map[key] = 9
    else:
        user_id_to_category_map[key] = 5
customer_purchase_power.median()
def get_customer_category(user_id):
    if user_id in user_id_to_category_map:
        return user_id_to_category_map[user_id]
    return 5
train["buying_power"] = list(map(lambda user_id: get_customer_category(user_id), train["User_ID"]))
test["buying_power"] = list(map(lambda user_id: get_customer_category(user_id), test["User_ID"]))

In [6]:
train["User_ID_MinPrice"] = train.groupby(['User_ID'])['Purchase'].transform('min')
userID_min_dict = train.groupby(['User_ID'])['Purchase'].min().to_dict()
test['User_ID_MinPrice'] = test['User_ID'].apply(lambda x:userID_min_dict.get(x,0))

train["User_ID_MaxPrice"] = train.groupby(['User_ID'])['Purchase'].transform('max')
userID_max_dict = train.groupby(['User_ID'])['Purchase'].max().to_dict()
test['User_ID_MaxPrice'] = test['User_ID'].apply(lambda x:userID_max_dict.get(x,0))

train["Product_ID_MinPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('min')
productID_min_dict = train.groupby(['Product_ID'])['Purchase'].min().to_dict()
test['Product_ID_MinPrice'] = test['Product_ID'].apply(lambda x:productID_min_dict.get(x,0))

train["Product_ID_MaxPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('max')
productID_max_dict = train.groupby(['Product_ID'])['Purchase'].max().to_dict()
test['Product_ID_MaxPrice'] = test['Product_ID'].apply(lambda x:productID_max_dict.get(x,0))

train["User_ID_Count"] = train.groupby(['User_ID'])['User_ID'].transform('count')
uID_count = train.groupby(['User_ID']).size().to_dict()
test['User_ID_Count'] = test['User_ID'].apply(lambda x:uID_count.get(x,0))

train["Product_ID_Count"] = train.groupby(['Product_ID'])['Product_ID'].transform('count')
pID_count = train.groupby(['Product_ID']).size().to_dict()
test['Product_ID_Count'] = test['Product_ID'].apply(lambda x:pID_count.get(x,0))

train["User_ID_MeanPrice"] = train.groupby(['User_ID'])['Purchase'].transform('mean')
userID_mean_dict = train.groupby(['User_ID'])['Purchase'].mean().to_dict()
test['User_ID_MeanPrice'] = test['User_ID'].apply(lambda x:userID_mean_dict.get(x,0))

train["Product_ID_MeanPrice"] = train.groupby(['Product_ID'])['Purchase'].transform('mean')
productID_mean_dict = train.groupby(['Product_ID'])['Purchase'].mean().to_dict()
test['Product_ID_MeanPrice'] = test['Product_ID'].apply(lambda x:productID_mean_dict.get(x,0))

train["Product_Cat1_MeanPrice"] = train.groupby(['Product_Category_1'])['Purchase'].transform('mean')
pc1_mean_dict = train.groupby(['Product_Category_1'])['Purchase'].mean().to_dict()
test['Product_Cat1_MeanPrice'] = test['Product_Category_1'].apply(lambda x:pc1_mean_dict.get(x,0))

train["Product_Cat2_MeanPrice"] = train.groupby(['Product_Category_2'])['Purchase'].transform('mean')
pc2_mean_dict = train.groupby(['Product_Category_2'])['Purchase'].mean().to_dict()
test['Product_Cat2_MeanPrice'] = test['Product_Category_2'].apply(lambda x:pc2_mean_dict.get(x,0))

train["Product_Cat3_MeanPrice"] = train.groupby(['Product_Category_3'])['Purchase'].transform('mean')
pc3_mean_dict = train.groupby(['Product_Category_3'])['Purchase'].mean().to_dict()
test['Product_Cat3_MeanPrice'] = test['Product_Category_3'].apply(lambda x:pc3_mean_dict.get(x,0))

userID_25p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['User_ID_25PercPrice'] = train['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))
test['User_ID_25PercPrice'] = test['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))

userID_50p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,50.)).to_dict()
train['User_ID_50PercPrice'] = train['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))
test['User_ID_50PercPrice'] = test['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))

userID_75p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['User_ID_75PercPrice'] = train['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))
test['User_ID_75PercPrice'] = test['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))

productID_25p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Product_ID_25PercPrice'] = train['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))
test['Product_ID_25PercPrice'] = test['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))

productID_50p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Product_ID_50PercPrice'] = train['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))
test['Product_ID_50PercPrice'] = test['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))

productID_75p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Product_ID_75PercPrice'] = train['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))
test['Product_ID_75PercPrice'] = test['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))

train["Product_Category_1_Count"] = train.groupby(['Product_Category_1'])['Product_Category_1'].transform('count')
pc1_count_dict = train.groupby(['Product_Category_1']).size().to_dict()
test['Product_Category_1_Count'] = test['Product_Category_1'].apply(lambda x:pc1_count_dict.get(x,0))

train["Product_Category_2_Count"] = train.groupby(['Product_Category_2'])['Product_Category_2'].transform('count')
pc2_count_dict = train.groupby(['Product_Category_2']).size().to_dict()
test['Product_Category_2_Count'] = test['Product_Category_2'].apply(lambda x:pc2_count_dict.get(x,0))

train["Product_Category_3_Count"] = train.groupby(['Product_Category_3'])['Product_Category_3'].transform('count')
pc3_count_dict = train.groupby(['Product_Category_3']).size().to_dict()
test['Product_Category_3_Count'] = test['Product_Category_3'].apply(lambda x:pc3_count_dict.get(x,0))

In [7]:
train_y = train["Purchase"]
train.drop(["Purchase"], axis=1, inplace=True)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(    
    train, train_y, test_size=0.05, random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()

# first we fit the scaler on the training dataset
scaler.fit(train)

# then we call the transform method to scale both the training and testing data
x_train_scaled = scaler.transform(train)
test_scaled=scaler.transform(test)

In [9]:
model = Sequential()
model.add(Dense(32, input_dim=train.shape[1], activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1)) # Output
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='mean_squared_error')
estp = EarlyStopping(monitor='val_loss', min_delta=0,patience=8, verbose=1, mode='auto',restore_best_weights=True)
model.fit(x_train_scaled,train_y,validation_split=0.15,shuffle='True',verbose=2,epochs=200, callbacks=[estp])

Epoch 1/200
14539/14539 - 20s - loss: 6961449.0000 - val_loss: 6125320.0000
Epoch 2/200
14539/14539 - 20s - loss: 6498350.0000 - val_loss: 6108585.5000
Epoch 3/200
14539/14539 - 20s - loss: 6415414.5000 - val_loss: 6108112.0000
Epoch 4/200
14539/14539 - 20s - loss: 6384537.5000 - val_loss: 6292367.0000
Epoch 5/200
14539/14539 - 20s - loss: 6354053.0000 - val_loss: 6033735.0000
Epoch 6/200
14539/14539 - 20s - loss: 6328991.5000 - val_loss: 6019710.5000
Epoch 7/200
14539/14539 - 20s - loss: 6309747.0000 - val_loss: 6012759.0000
Epoch 8/200
14539/14539 - 20s - loss: 6293335.0000 - val_loss: 5998164.0000
Epoch 9/200
14539/14539 - 20s - loss: 6273415.0000 - val_loss: 6025114.5000
Epoch 10/200
14539/14539 - 20s - loss: 6259729.5000 - val_loss: 6005425.5000
Epoch 11/200
14539/14539 - 20s - loss: 6240197.0000 - val_loss: 5991335.5000
Epoch 12/200
14539/14539 - 20s - loss: 6229860.5000 - val_loss: 5959888.0000
Epoch 13/200
14539/14539 - 21s - loss: 6217678.5000 - val_loss: 6058498.0000
Epoch 14

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                1440      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              33792     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 3

In [11]:
pred3 = model.predict(test_scaled)

In [12]:
df3 = pd.DataFrame(pred3, columns = ['Purchase'])

In [22]:
dl['Purchase']=pred3

In [ ]:
sum(n < 30 for n in dl["Purchase"].values.flatten())


In [23]:
dl.to_csv('deepfinal.csv',index=False)